In [ ]:
pip install deep_translator googletrans requests beautifulsoup4 eland elasticsearch huggingface-hub langchain tqdm torch requests sentence_transformers googletrans==4.0.0-rc1 reportlab


# Web Scraping con BeautifulSoup

Este cuaderno demuestra cómo realizar web scraping utilizando `requests` y `BeautifulSoup` para extraer el contenido principal de un artículo de una página web, traducir el contenido al español, mostrarlo en pantalla y, opcionalmente, guardarlo en un archivo PDF.

## Pasos:
1. **Enviar una solicitud HTTP**: Utilizamos `requests` para enviar una solicitud HTTP a la URL de la página web.
2. **Analizar el HTML**: Usamos `BeautifulSoup` para analizar el contenido HTML de la página web.
3. **Identificar y extraer el contenido**: Buscamos los elementos HTML que contienen el contenido principal del artículo (párrafos y encabezados).
4. **Dividir el contenido en fragmentos**: Para manejar las limitaciones de longitud de los servicios de traducción, dividimos el contenido en fragmentos más pequeños.
5. **Traducir el contenido**: Utilizamos `deep_translator` para traducir cada fragmento de texto al español.
6. **Mostrar el contenido traducido**: Unimos los fragmentos traducidos y mostramos el contenido completo en pantalla.
7. **Guardar el contenido en un PDF**: Si el usuario lo desea, se guarda el contenido traducido en un archivo PDF utilizando la biblioteca `reportlab`.

## Instalación de dependencias:
Ejecuta el siguiente comando para instalar las bibliotecas necesarias:

```bash
pip install deep_translator googletrans requests beautifulsoup4 eland elasticsearch huggingface-hub langchain tqdm torch requests sentence_transformers googletrans==4.0.0-rc1 reportlab


In [38]:
web = input ("Cual es la web a estudiar?")
# web = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6627879/"

Cual es la web a estudiar?https://blogs.elconfidencial.com/espana/una-cierta-mirada/2024-06-07/para-que-sirven-elecciones-europeas_3898196/


In [40]:
import requests
from bs4 import BeautifulSoup
from deep_translator import GoogleTranslator, exceptions
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
import textwrap

def beautifulsoup_web_scrape_url(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Verifica si la solicitud fue exitosa
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener la página web: {e}")
        return None

def extract_main_content(soup):
    # Definir posibles contenedores de contenido
    possible_containers = ['article', 'div', 'main', 'section']

    for tag in possible_containers:
        main_content = soup.find(tag)
        if main_content:
            paragraphs = main_content.find_all(['p', 'h2', 'h3'])
            if paragraphs:
                content_text = ""
                for section in paragraphs:
                    if section.name in ['h2', 'h3']:
                        content_text += f"## {section.get_text()}\n\n"
                    else:
                        content_text += f"{section.get_text()}\n\n"
                return content_text

    # Si no se encuentra un contenedor claro, buscar todos los párrafos
    paragraphs = soup.find_all('p')
    if paragraphs:
        content_text = ""
        for p in paragraphs:
            content_text += f"{p.get_text()}\n\n"
        return content_text

    return "No se pudo encontrar el contenido del artículo."

def translate_text(text, dest_language='es'):
    max_chunk_size = 5000  # Ajuste según sea necesario
    translator = GoogleTranslator(source='auto', target=dest_language)
    text_chunks = [text[i:i + max_chunk_size] for i in range(0, len(text), max_chunk_size)]
    translated_chunks = []

    for chunk in text_chunks:
        try:
            translated = translator.translate(chunk)
            translated_chunks.append(translated)
        except exceptions.TranslationNotFound as e:
            print(f"Error en la traducción del segmento: {e}")
            translated_chunks.append(chunk)  # Devolver el texto original del segmento en caso de error
        except Exception as e:
            print(f"Otro error en la traducción del segmento: {e}")
            translated_chunks.append(chunk)  # Devolver el texto original del segmento en caso de error

    return ' '.join(translated_chunks)

def save_as_pdf(content, filename):
    doc = SimpleDocTemplate(filename, pagesize=letter)
    styles = getSampleStyleSheet()
    story = []

    # Definir estilos personalizados
    custom_styles = {
        'Heading2': ParagraphStyle(
            'Heading2',
            parent=styles['Heading2'],
            fontSize=14,
            spaceAfter=12
        ),
        'BodyText': ParagraphStyle(
            'BodyText',
            parent=styles['BodyText'],
            fontSize=12,
            spaceAfter=12
        )
    }

    paragraphs = content.split('\n\n')
    for para in paragraphs:
        if para.startswith("## "):
            story.append(Paragraph(para[3:], custom_styles['Heading2']))
        else:
            story.append(Paragraph(para, custom_styles['BodyText']))
        story.append(Spacer(1, 12))

    doc.build(story)

# Variable web que contiene la URL
# web = "https://es.wikipedia.org/wiki/Python"  # Reemplaza esto con la URL que deseas analizar
soup = beautifulsoup_web_scrape_url(web)

if soup:
    article_text = extract_main_content(soup)
    if article_text:
        translate_option = input("¿Quieres traducir el contenido al español? (s/n): ").strip().lower()
        if translate_option in ['s', 'si']:
            translated_text = translate_text(article_text)
            print(translated_text)
        else:
            translated_text = article_text
            print(article_text)

        save_option = input("¿Quieres guardar el contenido en un PDF? (s/n): ").strip().lower()
        if save_option in ['s', 'si']:
            filename = input("Introduce el nombre del archivo (con .pdf al final): ").strip()
            if not filename.endswith('.pdf'):
                filename += '.pdf'
            save_as_pdf(translated_text, filename)
            print(f"El contenido ha sido guardado en {filename}.")
    else:
        print("No se pudo extraer el contenido del artículo.")
else:
    print("Error al obtener la página web.")


¿Quieres traducir el contenido al español? (s/n): n
##    Una Cierta Mirada   

##  Por     Ignacio Varela  

## Lo que se decide en las instituciones europeas afecta a nuestras vidas en mayor medida que todo lo que sale de las de aquí

 Se han realizado cuatro elecciones europeas en el siglo XXI, esta será la quinta. Tres de ellas (2004, 2009 y 2014) se celebraron en solitario, sin coincidir con ninguna otra votación. La participación en esos tres casos fue la siguiente: 45,1%, 44,9% y 43,8%. Una línea descendente que sugiere que, cuando la europea es la única elección que nos convoca, casi seis de cada diez adultos con derecho de voto se desentienden del evento. Es muy verosímil que eso suceda también en esta ocasión. 

 La participación subió al 60,7% en 2019 por un único motivo: las elecciones europeas coincidieron con las municipales en toda España y con las autonómicas en 13 comunidades. Además, en la mayoría de los territorios el resultado en la urna europea reprodujo el de la m